Fit each transit with a GP + transit. Lock all parameters except rprs and ars.

This code is only minimally modified from https://github.com/AnaLopezMurillo/TaurusTTVs

If you find this code helpful in your research, please cite Barber et al. in prep and Lopez Murillo et al. in prep.

In [ ]:
import lightkurve as lk
from lightkurve import LightCurveCollection
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import batman
from scipy.optimize import curve_fit
import emcee
from scipy.optimize import minimize
import re
from scipy import stats
import corner
import matplotlib.gridspec as gridspec
import celerite2
from celerite2 import terms
import scipy
%matplotlib inline

In [ ]:
# DATA UPLOAD
lc_path = './LCs/extractedLC.txt'

df = pd.read_csv(lc_path, delimiter=" ")
time_array = df["time"].to_numpy()
data = df["flux"].to_numpy()
ferr = df["flerr"].to_numpy()

In [ ]:
## filenames:
chainfile = './LCs/emceechain-56658270.npy'
probfile = './LCs/emceeprob-56658270.npy'

## read in the npy file
chain = np.load(chainfile) ## contains the parameters at each step
prob = np.load(probfile)   ## contains the probabilities (likelihoods) at each step

## flatten the chain/probs + remove burn in
ndim = (np.shape(chain))[2]
nburnin= chain.shape[1]//3
samples=chain[:,nburnin:,:].reshape((-1,ndim))
probs=prob[:,nburnin:].reshape((-1))

In [ ]:
plt.plot(time_array, data)

In [ ]:
# Running Median Function - used for plotting
def runmed(a,b,size): # a= time array, b= flux array, size=size of bin
    meda = []
    medb = []
    for i in np.arange(np.min(a),np.max(a),size):
        l = np.where((a > i) & (a<(i+size)))
        meda.append(np.median(a[l]))
        medb.append(np.median(b[l]))
    return meda,medb

In [ ]:
## get highest likelihood set of parameters:
l = np.squeeze((np.where(probs == np.max(probs))))[1] ## many points may have equal likelihood, it doesn't matter which you select
best = samples[l,:]

logP = best[7]
logamp = best[8]
logQ1 = best[9]
logQ2 = best[10]
logmix = best[11]

t0 = best[0]
per = best[1]
rp = best[2]
b = best[3]
rho = best[4]
q1 = best[5]
q2 = best[6]
w = 0.
ecc = 0.0

ars = 215.*rho**(1./3.)*(per/365.25)**(2./3.)
inc = np.arccos(b/ars)*180./np.pi  

window = 1.0
t0_guess = t0

In [ ]:
fullDepthPosterior = samples[:,2]

In [ ]:
# transit model
def transit(t, rp, a):
    params = batman.TransitParams()
    params.t0 = t0_guess                 
    params.per = per             
    params.rp = rp                 
    params.a = a                
    params.inc = inc                
    params.ecc = ecc                  
    params.w = w                   
    params.u = [0.29, 0.329]          
    params.limb_dark = "quadratic"  

    m = batman.TransitModel(params, t)
    flux = m.light_curve(params)      
    return flux 

# MCMC functions
def set_params(theta, time, gp, err):
    rp, a, logsigma, logrho, logQ = theta
    gp.kernel = terms.SHOTerm(sigma=np.exp(logsigma), rho=np.exp(logrho), Q=np.exp(logQ)) 
    gp.compute(time, yerr=err, quiet=True)
    return gp

def ln_likelihood(theta, time, gp, data, err):
    rp, a, logsigma, logrho, logQ = theta
    try: 
        gp = set_params(theta, time, gp, err)
    except ZeroDivisionError:
        return -np.inf
    model = transit(time, rp, a)
    return gp.log_likelihood(data-model)


def ln_prior(theta):
    rp, a, logsigma, logrho, logQ = theta
    sigma = np.exp(logsigma)
    rho = np.exp(logrho)
    Q = np.exp(logQ)
    
    if rp < .0676 - (0.0024*10) or rp > .0676 + (0.0024*10) or a > 20 or a < 0 or (Q == 0) or (rho == 0) or (sigma == 0):
        return -np.inf
   
    sigma_sigma2 = 20.0
    lp1 = -0.5*np.sum((sigma)**2/sigma_sigma2 + np.log(2*np.pi*sigma_sigma2))
    sigma_q2 = 20.0
    lp3 = -0.5*np.sum((Q)**2/sigma_q2 + np.log(2*np.pi*sigma_q2))
    return lp1+lp3

def ln_probability(theta, time, gp, data, err):
    lp = ln_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    lnlike = ln_likelihood(theta, time,gp,data,err)
    if not np.isfinite(lnlike):
        return -np.inf
    return lp + lnlike

def initWalkers(theta_0, ndim, nwalkers):
    
    perturbation = [0.001, 0.001, 0.001, 0.001, 0.001]
    rng = np.random.default_rng()
    
    pos = []
    
    while len(pos) < nwalkers:
        p = theta_0 + perturbation*rng.standard_normal(ndim)
        
        if np.isfinite(ln_prior(p)):
            pos.append(p)
    
    return pos

### Handle the TESS data 

In [ ]:
tt = []
chi2 = []

depths = []
ars = []
depthUncerts = []
axesUncerts = []
depthChains = []
arsChains = []

transit_num = -1

rho_0 = 10**logP
logQ_0 = .7
t0_guess = t0

for i in np.arange(-1000,3000, 1):
    t0_guess = t0+(per*i)
    rho = 1.4
    logQ = .7

    initial = [0.0673, 11.81, -4,  np.log(rho_0),  logQ_0]

    window = 0.7

    ll = np.where((time_array > t0_guess - window) & (time_array < t0_guess + window))
    ll2 = np.where((time_array > t0_guess - 0.1) & (time_array < t0_guess + 0.1))

    if (np.size(ll) > 10) and (np.size(ll2) > 0):
        print(t0_guess)
        y = np.array(data[ll])
        t = np.array(time_array[ll])
        err = np.array(ferr[ll])

        transit_num+=1

        term2 = terms.SHOTerm(sigma=np.exp(initial[2]), rho=np.exp(initial[3]), Q=np.exp(initial[4]))
        kernel = term2        
        
        gp = celerite2.GaussianProcess(kernel, mean=0.0)
        gp.compute(t, yerr=err)
        print("Initial log likelihood: {0}".format(gp.log_likelihood(y)))

        ln_probability(initial,t,gp,y,err)

        # set up MCMC sampler
        max_n = 1000
        nwalkers = 10
        initial_positions = initWalkers(initial, len(initial), nwalkers)

        sampler = emcee.EnsembleSampler(nwalkers, len(initial), ln_probability, args=(time_array[ll], gp, data[ll], err), threads=8)

        index = 0
        autocorr = np.empty(max_n)
        autcorrreq = 50
        autcorr_change_frac = 0.1
        old_tau = np.inf

        for sample in sampler.sample(initial_positions, iterations=max_n, progress=True):
            # Check convergence every 1000 steps
            if sampler.iteration % 1000:
                continue
            
            # Compute the autocorrelation time so far
            # Using tol=0 means that we'll always get an estimate even if it isn't trustworthy
            tau = sampler.get_autocorr_time(tol=0)
            autocorr[index] = np.mean(tau)
            index += 1

            # Check convergence
            converged = np.all(tau * autcorrreq < sampler.iteration)
            converged &= np.all(np.abs((old_tau - tau) / tau) < autcorr_change_frac)
            if converged:
                break
            old_tau = tau

        ## burn and flat samples finding
        burn = sampler.iteration//5
        flat_samples = sampler.get_chain(discard=burn, thin=3, flat=True)
        mcmc = np.percentile(flat_samples[:, 0], [16, 50, 84])

        samples = sampler.get_chain(discard=burn)
        flats = np.concatenate(samples)

        # walker plot
        ndim = len(initial)
        labels = [r'$rp/rs$', r'$a/rs$', r'$\ln{\sigma}$',r'$\ln{\rho}$',r'$\ln{Q}$']
        fig, axes = plt.subplots(ndim, figsize=(10, 7), sharex=True, facecolor="white")
        for i in range(ndim):
            ax = axes[i] 
            ax.plot(samples[:, :, i], "k", alpha=0.3)
            ax.set_xlim(0, len(samples))
            ax.set_ylabel(labels[i])
            ax.yaxis.set_label_coords(-0.1, 0.5)
        plt.show()

        # corner plot
        flat_samples = sampler.get_chain(discard=burn, flat=True)
        fig = corner.corner( 
            flat_samples,show_titles=True,labels=labels,quantiles=(0.16, 0.84),
            fill_contours=True, plot_datapoints=False,title_kwargs={"fontsize": 9},title_fmt='.3f',
            hist_kwargs={"linewidth": 2.5},levels=[(1-np.exp(-0.5)),(1-np.exp(-2)),(1-np.exp(-4.5))],
            title_quantiles=[0.16, 0.5, 0.84]
        )
        plt.show()

        # data model plot
        fig, axes = plt.subplots(1, figsize=(10, 7), sharex=True)
        result = np.median(flat_samples,axis=0)
        set_params(result, t, gp, err)
        model = transit(t,result[0], result[1])
        mu, variance = gp.predict(y-model, t, return_var=True)
        sigma = np.sqrt(variance)
        axes.plot(t,model,color='r')

        marker = "."
        if (np.size(ll) < 2500):
            marker="o"
        axes.scatter(t,y-mu, marker=marker, alpha=0.2, color='r')
        if np.size(ll) > 100:
            bint, binfl = runmed(t, y - mu, 0.33/24.)
            plt.scatter(bint, binfl, marker='o', alpha=0.5, color='b')
        plt.title('Transit #' + str(transit_num))
        plt.show()

        print(" ")
        print(" ")

        tt.append(t0_guess)

        depths.append(result[0])
        ars.append(result[1])
        depthUncerts.append(np.std(flats[:,0]))
        axesUncerts.append(np.std(flats[:,1]))
        
        depthChains.append(flat_samples[:,0])
        arsChains.append(flat_samples[:,1])
